In [0]:
import requests
from datetime import datetime
import logging
import urllib3

import sailpoint
import sailpoint.beta
import sailpoint.v2024
from sailpoint.configuration import (Configuration, ConfigurationParams)
from sailpoint.paginator import Paginator

from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType, MapType, DateType, ArrayType

In [0]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create console handler and set level to info
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# Create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Add formatter to ch
ch.setFormatter(formatter)

# Add ch to logger
logger.addHandler(ch)

In [0]:
tenant = 'devrel-ga-5668'
client_name = 'sailpoint-readall-client'
secret_name = 'sailpoint-readall-secret'

In [0]:
logger.info('Defining API credentials')
api_url = f'https://{tenant}.api.identitynow-demo.com'
token_url = f'https://{tenant}.api.identitynow-demo.com/oauth/token'
client_id = dbutils.secrets.get(scope='sailpoint-devrel', key=client_name)
client_secret = dbutils.secrets.get(scope='sailpoint-devrel', key=secret_name)

In [0]:
logger.info('Configuring SailPoint SDK')
configurationParams = ConfigurationParams()
configurationParams.base_url = api_url
configurationParams.token_url = token_url
configurationParams.client_id = client_id
configurationParams.client_secret = client_secret
configuration = Configuration(configurationParams)
configuration.experimental = True
configuration.retries = urllib3.Retry(
    total=5,  # Total number of retries
    backoff_factor=10,  # Backoff factor for retry delay
    status_forcelist=[429, 500, 502, 503, 504],  # HTTP status codes to retry on
    allowed_methods=["GET"]  # HTTP methods to retry
)

In [0]:
async def get_form_instances():
    logger.info('Retrieving form_instances...')
    try:
        all_forms = []
        with sailpoint.beta.ApiClient(configuration) as api_client:
            api_instance = sailpoint.beta.CustomFormsApi(api_client)
            all_forms = api_instance.search_form_instances_by_tenant()
            logger.info(f'Example form: {all_forms[2]}')
            logger.info(f'Retrieved {len(all_forms)} forms')
            return all_forms
    except Exception as e:
        logger.error(f'Error Retrieving forms: {e}')
        raise Exception(e)

In [0]:
all_forms = await get_form_instances()
logger.info(f'Total Retrieved Forms: {len(all_forms)}')

In [0]:
object_schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("type", StringType(), True),
    ]
)

form_instance_schema = StructType([
    StructField("id", StringType(), True),
    StructField("created", TimestampType(), True),
    StructField("modified", TimestampType(), True),
    StructField("created_by", object_schema, True),
    StructField("expire", StringType(), True),
    StructField("form_conditions", ArrayType(StringType(), True), True),
    StructField("form_definition_id", StringType(), True),
    StructField("form_data", MapType(StringType(), StringType()), True),
    StructField("form_errors", ArrayType(StringType(), True), True),
    StructField("form_input", MapType(StringType(), StringType()), True),
    StructField("recipients", ArrayType(object_schema, True), True),
    StructField("stand_alone_form", BooleanType(), True),
    StructField("stand_alone_form_url", StringType(), True),
    StructField("state", StringType(), True),
])

In [0]:
try:
    logger.info('Creating data frame...')
    forms_df = spark.createDataFrame(all_forms, form_instance_schema)
    today = datetime.utcnow().date()
    forms_df = forms_df.withColumn('day', lit(today))

    logger.info('Writing to delta...')
    forms_df.write \
        .format('delta') \
        .mode('append') \
        .partitionBy('day') \
        .saveAsTable(f'`dunker_databricks_space`.sailpoint.form_instances')

except Exception as e:
    logger.error(f'Error in creating data frame or writing to delta: {e}')